<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>

## Assessment: Computer Vision for Industrial Inspection ##
In this notebook, you will utilize what you've learned in this course to complete an assessment. The assessment has been divided into a couple of steps to guide your development. You will be graded based on the performance of your deep learning model. Note that this coding portion does not give partial credit - it shows up as either 0 or 60 points. 
<table border="1" class="dataframe" align='left'>  <thead>    <tr style="text-align: right;">      <th>Step</th>      <th></th>      <th>Points</th>    </tr>  </thead>  <tbody>    <tr>      <td>0. The Problem</td>      <td></td>      <td></td>    </tr>    <tr>      <td>1. Data Curation</td>      <td></td>      <td></td>    </tr>    <tr>      <td>2. Prepare TAO Experiment</td>      <td></td>      <td></td>    </tr>    <tr>      <td>3. Model Training</td>      <td></td>      <td></td>    </tr>    <tr>      <td>4. Model Evaluation</td>      <td></td>      <td>60</td>    </tr></tbody></table>

<p><img src='images/ml_workflow.png' width=720></p>

### Step 0: The Problem ###
In this course, we made a binary classifier for the true/false defective units in our printed circuit board assembly dataset. For the asssesment we are asking you to create a model over the same dataset, but with a different purpose. Before we dealt with `capacitors` only, but the reference circuit boards have other components as well. In particular, we are interested in classifying the following four components based on their images: 

<p><img src='images/assessment_samples.png' width=720></p>

The component types are marked by the first letter(s) of component identification numbers. 
* **C** - Capacitor
* **Q** - Transistor
* **R** - Resistor
* **U** - Integrated Circuit

Your task is to train and evaluate a classifier that accurately labels the four component types. We recommend using `VGG19` as the architecture in the spirit of experimenting with different backbones. 

**Instructions**: <br> 
0.1 Execute the below cell to import dependencies <br>
0.2 Execute the cell below to unzip and load data

In [1]:
# 0.1
# DO NOT CHANGE THIS CELL
# import dependencies
import os
import warnings
import pandas as pd
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import math
import shutil
warnings.filterwarnings("ignore")

In [2]:
# 0.2
# DO NOT CHANGE THIS CELL
# unzip
!unzip -qq data/viz_BYD_new.zip -d data

# remove zip file
!rm data/viz_BYD_new.zip

### Step 1: Data Curation ###
The first step is to curate the data. 

**Instructions**: <br>
1.1 Execute the below cell to load the data into a `DataFrame` and preview. <br>
1.2 Modify the `<FIXME>` only and execute the cell below to filter and keep only normal images as potential defective images may include missing parts. <br>
1.3 Modify the `<FIXME>` only and execute the cell below to filter and keep only relevant images, i.e. `C`, `Q`, `R`, or `U`. <br>
1.4 Execute the cell below to check the sample size. 

In [3]:
# 1.1
# DO NOT CHANGE THIS CELL
# load from pcba_df.csv
pcba_df=pd.read_csv('pcba_df.csv')
pcba_df.head()

,true_defect,defect_img_path,date,board,comp_id,img_shape,defect_image_name,comp_type
0,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026324/...,908,423318026324,C1090,"(54, 27, 3)",D0_C1090.jpg,C
1,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026269/...,908,423318026269,C1090,"(54, 27, 3)",D1_C1090.jpg,C
2,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026261/...,908,423318026261,L503,"(347, 418, 3)",D1_L503.jpg,L
3,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026523/...,908,423318026523,L503,"(347, 418, 3)",D1_L503.jpg,L
4,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026523/...,908,423318026523,C1090,"(54, 27, 3)",D1_C1090.jpg,C


In [4]:
# 1.2
verified_df=pcba_df[pcba_df['true_defect']=='notdefect']
verified_df.head()

,true_defect,defect_img_path,date,board,comp_id,img_shape,defect_image_name,comp_type
0,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026324/...,908,423318026324,C1090,"(54, 27, 3)",D0_C1090.jpg,C
1,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026269/...,908,423318026269,C1090,"(54, 27, 3)",D1_C1090.jpg,C
2,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026261/...,908,423318026261,L503,"(347, 418, 3)",D1_L503.jpg,L
3,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026523/...,908,423318026523,L503,"(347, 418, 3)",D1_L503.jpg,L
4,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026523/...,908,423318026523,C1090,"(54, 27, 3)",D1_C1090.jpg,C


In [5]:
# 1.3
cqru_df=verified_df[verified_df['comp_type'].isin(['C','Q','R','U'])]
cqru_df.head()

,true_defect,defect_img_path,date,board,comp_id,img_shape,defect_image_name,comp_type
0,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026324/...,908,423318026324,C1090,"(54, 27, 3)",D0_C1090.jpg,C
1,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026269/...,908,423318026269,C1090,"(54, 27, 3)",D1_C1090.jpg,C
4,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026523/...,908,423318026523,C1090,"(54, 27, 3)",D1_C1090.jpg,C
5,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026331/...,908,423318026331,C1090,"(54, 27, 3)",D1_C1090.jpg,C
6,notdefect,/dli/task/data/AOI_DL_data_0908/0423318026211/...,908,423318026211,C1090,"(53, 27, 3)",D1_C1090.jpg,C


In [6]:
# 1.4
# DO NOT CHANGE THIS CELL
# check sample size
cqru_df.groupby('comp_type').size()

comp_type
C    1804
Q     775
R     690
U    1388
dtype: int64

### Step 2: Prepare TAO Experiment ###
Next we will prepare for the TAO experiment. 

**Instructions**: <br>
2.1 Execute the below cell to set environment variables. <br>
2.2 Execute the cell below to map up local directories to the TAO docker. <br>
2.3 Execute the cell below to use the `ngc registry mode list` command that lists all available `classification` models. <br>
2.4 Modify the `<FIXME>` only and execute the cell below to download the `VGG19` pre-trained weights. <br>
2.5 Execute the cell below to view the pre-trained model. <br>
2.6 Execute the cell below to create the required data directories. <br>
2.7 Execute the cell below to create a 70%/30% split for train and val. <br>
2.8 Execute the cell below to copy data from the source to the TAO experiment folder. 

In [7]:
# 2.1
# DO NOT CHANGE THIS CELL
# set environment variables
%set_env KEY=my_model_key

%set_env LOCAL_PROJECT_DIR=/dli/task/tao_project
%set_env LOCAL_DATA_DIR=/dli/task/tao_project/data
%set_env LOCAL_SPECS_DIR=/dli/task/tao_project/spec_files
os.environ["LOCAL_EXPERIMENT_DIR"]=os.path.join(os.getenv("LOCAL_PROJECT_DIR"), "classification")

%set_env TAO_PROJECT_DIR=/workspace/tao-experiments
%set_env TAO_DATA_DIR=/workspace/tao-experiments/data
%set_env TAO_SPECS_DIR=/workspace/tao-experiments/spec_files
os.environ['TAO_EXPERIMENT_DIR']=os.path.join(os.getenv("TAO_PROJECT_DIR"), "classification")

# make the data directory
!mkdir -p $LOCAL_DATA_DIR

env: KEY=my_model_key
env: LOCAL_PROJECT_DIR=/dli/task/tao_project
env: LOCAL_DATA_DIR=/dli/task/tao_project/data
env: LOCAL_SPECS_DIR=/dli/task/tao_project/spec_files
env: TAO_PROJECT_DIR=/workspace/tao-experiments
env: TAO_DATA_DIR=/workspace/tao-experiments/data
env: TAO_SPECS_DIR=/workspace/tao-experiments/spec_files


In [8]:
# 2.2
# DO NOT CHANGE THIS CELL
# map local directories to the TAO docker
mounts_file = os.path.expanduser("~/.tao_mounts.json")

drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["TAO_SPECS_DIR"]
            },
            # Mapping the data directory.
            {
                "source": os.environ["LOCAL_DATA_DIR"],
                "destination": os.environ["TAO_DATA_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# writing the mounts file
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [9]:
# 2.3
# DO NOT CHANGE THIS CELL
!ngc registry model list nvidia/tao/pretrained_classification:*

[{
    "accuracyReached": 77.56,
    "batchSize": 1,
    "createdByUser": "n90fe0en2gvll5957fel7u75sg",
    "createdDate": "2021-08-18T20:15:41.146Z",
    "description": "",
    "gpuModel": "V100",
    "memoryFootprint": "153.7",
    "numberOfEpochs": 80,
    "status": "UPLOAD_COMPLETE",
    "totalFileCount": 1,
    "totalSizeInBytes": 161183816,
    "versionId": "vgg19"
},{
    "accuracyReached": 77.17,
    "batchSize": 1,
    "createdByUser": "n90fe0en2gvll5957fel7u75sg",
    "createdDate": "2021-08-18T20:15:13.258Z",
    "description": "",
    "gpuModel": "V100",
    "memoryFootprint": "113.2",
    "numberOfEpochs": 80,
    "status": "UPLOAD_COMPLETE",
    "totalFileCount": 1,
    "totalSizeInBytes": 118655144,
    "versionId": "vgg16"
},{
    "accuracyReached": 65.13,
    "batchSize": 1,
    "createdByUser": "n90fe0en2gvll5957fel7u75sg",
    "createdDate": "2021-08-18T20:18:00.942Z",
    "description": "",
    "gpuModel": "V100",
    "memoryFootprint": "6.5",
    "numberOfEpochs": 

In [15]:
# 2.4
# create directory to store the pre-trained model
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_vgg19/

# download the pre-trained vgg19 model from NGC
!ngc registry model download-version nvidia/tao/pretrained_classification:vgg19 --dest $LOCAL_EXPERIMENT_DIR/pretrained_vgg19

{
    "download_end": "2024-05-21 07:07:46.634745",
    "download_start": "2024-05-21 07:07:42.628996",
    "download_time": "4s",
    "files_downloaded": 1,
    "local_path": "/dli/task/tao_project/classification/pretrained_vgg19/pretrained_classification_vvgg19",
    "size_downloaded": "142.58 MB",
    "status": "Completed",
    "transfer_id": "pretrained_classification_vvgg19"
}


<p><img src='images/tip.png' width=720></p>

We designated the model to be downloaded to `tao_project/classification/pretrained_vgg19`, which is mapped to `/workspace/tao-experiments/classification/pretrained_vgg19` in the TAO container based on the mapping of `LOCAL_EXPERIMENT_DIR` to `TAO_EXPERIMENT_DIR`. Looking at the `local_path` and `transfer_id` keys of the output JSON, we can gather that the path of the pre-trained model should be in the `tao_project/classification/pretrained_vgg19/pretrained_classification_vvgg19` directory. When referencing paths for the TAO Toolkit, it's important to use paths based on the TAO container. In this case it would be `/workspace/tao-experiments/classification/pretrained_vgg19/pretrained_classification_vvgg19`. 

In [16]:
# 2.5
# DO NOT CHANGE THIS CELL
!ls -al tao_project/classification/pretrained_vgg19/pretrained_classification_vvgg19

total 157416
drwx------ 2 root root      4096 May 21 07:07 .
drwxr-xr-x 3 root root      4096 May 21 07:07 ..
-rw------- 1 root root 161183816 May 21 07:07 vgg_19.hdf5


In [17]:
# 2.6
# DO NOT CHANGE THIS CELL
# remove existing data from previous experiment (if any)
!rm -rf $LOCAL_DATA_DIR/*

!mkdir -p $LOCAL_DATA_DIR/train/Q
!mkdir -p $LOCAL_DATA_DIR/train/C
!mkdir -p $LOCAL_DATA_DIR/train/U
!mkdir -p $LOCAL_DATA_DIR/train/R
!mkdir -p $LOCAL_DATA_DIR/val/Q
!mkdir -p $LOCAL_DATA_DIR/val/C
!mkdir -p $LOCAL_DATA_DIR/val/U
!mkdir -p $LOCAL_DATA_DIR/val/R

In [18]:
# 2.7
# DO NOT CHANGE THIS CELL
# set default as training set
cqru_df['data_set']='train'

# sample 30% and set as validation set
val_set=cqru_df.groupby('comp_type', group_keys=False).apply(lambda x: x.sample(frac=0.3))
cqru_df.loc[val_set.index, 'data_set']='val'
cqru_df.groupby(['data_set', 'comp_type']).size()

data_set  comp_type
train     C            1263
          Q             543
          R             483
          U             972
val       C             541
          Q             232
          R             207
          U             416
dtype: int64

In [19]:
# 2.8
# DO NOT CHANGE THIS CELL
# iterate through the DataFrame and copy images
for idx, row in cqru_df.iterrows(): 
    shutil.copyfile(row['defect_img_path'], f"{os.environ['LOCAL_DATA_DIR']}/{row['data_set']}/{row['comp_type']}/{row['date']}_{row['board']}_{row['defect_image_name']}")

### Step 3: Model Training ###
The next step is to modify the configuration file that will be used for `train`. You can create a new text file for this purpose manually and start from scratch or use the [template provided](tao_project/spec_files/vgg19/config.txt). You can also refer to sample applications and configuration files [here](https://docs.nvidia.com/tao/tao-toolkit/text/image_classification.html). 

**Instructions**: <br>
3.1. Open and review the [configuration file](tao_project/spec_files/vgg19/config.txt). <br>
3.2. Modify the `<FIXME>`s only in the configuration file with the correct values and **save changes**. We recommend starting with a very low epoch count (e.g. 5) in the interest of time as each epoch can take ~100s to complete. <br>
* Recall that you don't need the `eval_config` section of the configuration file, which requires you to know the trained model path. Of course, if you know where you would like to place the model, you can go ahead and complete this section. 
3.3 Execute the cell below to initiate model training. 

<p><img src='images/tip.png' width=720></p>

Based on how NGC names the pre-trained model downloaded, we should use `/workspace/tao-experiments/classification/pretrained_vgg19/pretrained_classification_vvgg19/resnet_19.hdf5` to reference the pre-trained model. Furthermore, we can choose where to store the trained model - in this case we use `/workspace/tao-experiments/classification/vgg19` inside of the TAO container, which is mapped to `tao_project/classification/vgg19` in our local drive. Furthermore, the trained model name will follow the format `<model_arch>_0<last_epoch_count>.tlt`, unless specified otherwise. Therefore we should use `/workspace/tao-experiments/classification/vgg19/weights/vgg_XXX.tlt`, depending on `n_epochs` in the model configuration. 

In [21]:
# 3.2
# DO NOT CHANGE THIS CELL
# train model
!tao classification train -e $TAO_SPECS_DIR/vgg19/config.txt \
                          -r $TAO_EXPERIMENT_DIR/vgg19 \
                          -k $KEY

2024-05-21 07:15:50,398 [INFO] root: Registry: ['nvcr.io']
2024-05-21 07:15:50,578 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.5-py3
Using TensorFlow backend.
Using TensorFlow backend.
2024-05-21 07:15:57,894 [INFO] __main__: Loading experiment spec at /workspace/tao-experiments/spec_files/vgg19/config.txt.

2024-05-21 07:15:57,898 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/train.py:281: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2024-05-21 07:15:57,898 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/makenet/scripts/train.py:290: The name tf.Session is de

### Step 4: Model Evaluation ###
The last step for the assessment is to `evaluate` the model through the `eval_config` section of the configuration file. Once completed, you should submit the output log for assessment. 

**Instructions**: <br>
4.1 Review the [configuration file](tao_project/spec_files/vgg19/config.txt) and modify the `eval_config` section if needed. <br>
4.2 Execute the below cell to evaluate the model. A log file will be generated through the use of the `--log_file` option, which will be used for grading purposes. <br>
4.3 Execute the cell below to submit the log for assessment. 

In [22]:
# 4.1
# DO NOT CHANGE THIS CELL
# evaluate the model using the same validation set as training
!tao classification evaluate -e $TAO_SPECS_DIR/vgg19/config.txt\
                             -k $KEY \
                             --log_file $TAO_PROJECT_DIR/log_file.txt

2024-05-21 07:26:47,914 [INFO] root: Registry: ['nvcr.io']
2024-05-21 07:26:48,078 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-tf:v3.21.11-tf1.15.5-py3
Using TensorFlow backend.
2024-05-21 07:27:41,917 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


Click [here](tao_project/log_file.txt) to view the log file. 

In [23]:
# 4.2
# DO NOT CHANGE THIS CELL
!cp $LOCAL_PROJECT_DIR/log_file.txt my_assessment/log_file.txt

### Grade Your Code ###
If you have trained the model and completed model evaluation successfully, save changes to the notebook and revisit the webpage where you launched this interactive environment. Click on the "**ASSESS TASK**" button as shown in the screenshot below. Doing so will give you credit for this part of the lab that counts towards earning a certificate of competency for the entire course.
<p><img src='images/credit.png' width=720></p>

<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>